# Workflows Guide

The [Workflows API](https://docs.descarteslabs.com/guides/workflows.html) provides a way to easily construct, execute, and visualize geospatial analysis pipelines.  It provides an abstraction layer on top of remote sensing data so that you can spend less time managing individual images and array operations and instead focus on your models & analysis.

You can run the following cells using `Shift-Enter`.

### Proxy Objects

At the base of the Workflows API there are a set of objects called `Proxy Objects`, which allow you to lazily construct a set of operations to later be executed on our backend. For example, the `Image` proxy object doesn't actually contain any image data itself, but instead references a image that will be constructed on demand at a future point in time.  The most important objects to understand to get started are the `ImageCollection`, `Image`, and `GeoContext` proxy types.

An `ImageCollection` object represents a stack (usually across time) of images, that are typically constructed using
the `ImageCollection.from_id` method. For example, to define a 5-day stack of Sentinel-2 Imagery, one would write

In [ ]:
# keep logging quiet
import logging; logging.getLogger().setLevel(logging.INFO); logging.captureWarnings(True)

In [ ]:
import descarteslabs.workflows as wf
s2_collection = wf.ImageCollection.from_id("sentinel-2:L1C", start_datetime='2019-12-10', end_datetime='2019-12-16')
s2_collection

This `s2_collection` variable does not actually contain any image data or metadata. It does, however, define the search criteria for the image data. In this case it defines a period of time from December 10th to 16th, 2019. It does not yet specify over which area of interest to evaluate the query, which we'll discuss shortly. 

While an `ImageCollection` represents a set of potentially overlapping images, an `Image` represents a single image which is represented by a 3D array of (band, horizonal-spatial, vertical-spatial). In the case of a 3-band image this might represent a red-green-blue standard image. An `Image` object can be derived from an `ImageCollection` through any one of the available reduction operations such as 'min', 'mean', 'max', 'median', etc. For example a minimum `Image` derived from the `s2_collection` object can be constructed with

In [ ]:
s2_image = s2_collection.min(axis='images')
s2_image

This `s2_image` variable is now an `Image` object. In order to realize this image over some area of the Earth, you'll next need to specify a `GeoContext`, which is the combination of a geometry (representing the area of interest), a coordinate system, and a resolution. A `GeoContext` can be constructed in a number of ways. If you are used to using the DLTile system in the `Scenes`, `Raster`, and `Metadata` APIs, you might use something like `wf.GeoContext.from_dltile_key()`. In this case, we'll construct an area of interest from a GeoJSON definition.  First we create a `workflows.Geometry` object from the GeoJSON dictionary.

In [ ]:
geometry = wf.Geometry.from_geojson({
        "type": "Polygon",
        "coordinates": [
          [
            [
              -102.78739929199219,
              36.0513195750255
            ],
            [
              -102.58415222167969,
              36.0513195750255
            ],
            [
              -102.58415222167969,
              36.16836821871061
            ],
            [
              -102.78739929199219,
              36.16836821871061
            ],
            [
              -102.78739929199219,
              36.0513195750255
            ]
          ]
        ]
      })

Next we create the `GeoContext` using the above geometry along with a coordinate system / resolution pair. In this case we'll use Web Mercator (ESPG:3857) for convenience, but for analysis you will likely want to use an appropriate equal-area projection.  We'll choose 20 meters as the resolution.

In [ ]:
geoctx = wf.GeoContext(geometry=geometry, crs='EPSG:3857', resolution=20.0)

We can now combine our `geoctx` and `s2_image` objects to evaluate our basic Workflow. After you execute the following cell, you should see information from the Workflows backend such as what stage of processing is complete.  The result is an [`ImageResult`](https://docs.descarteslabs.com/descarteslabs/workflows/docs/results.html#descarteslabs.workflows.results.ImageResult) object, which we store in the `image_result` variable. Here we're also going to choose the bands that we want and scale them from 0-40% reflectance. See the docstrings of `.pick_bands` and `.scale_values` for more info.

In [ ]:
image_result = s2_image.pick_bands("red green blue").scale_values(0, 1, 0.0, 0.4).compute(geoctx=geoctx)

An `ImageResult` object has a few important attributes. First, there is a `.ndarray`, which is a Numpy masked-array that contains the actual image data.  It could be displayed with, for example, Matplotlib. Note that the ndarray is ordered by the band, then spatial dimensions, so some plotting routines require a transpose.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[12,12])
plt.imshow(image_result.ndarray.transpose((1,2,0)))

This first part of the guide has covered some of the basic object types and methods that you might start using to build a model around geospatial data. For more info on the additional types, see the [API Documentation](https://docs.descarteslabs.com/descarteslabs/workflows/readme.html) or [Guide](https://docs.descarteslabs.com/guides/workflows.html). Next we'll dive into a few more examples around visualizations, reductions, and parameters.

### Tile-based Visualizations

While visualizing individual areas of interest is already useful, many models and applications may need to be developed across different types of locations. To ease the development of models that may even be applied globally, the Workflows API provides a tile server that can be used to visualize results on a navigable map.

In the context of the Jupyter ecosystem, we've chosen to integrate with the [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/) package, which provides a bridge between jupyter notebooks and the javascript [leaflet.js](https://leafletjs.com/) library. 

Integration is simple. Any Workflows `Image` object can be visualized on a map using the `.visualize(...)` method. We refer to each `Image` as a different "Layer" of the map, and a map can have multiple layers associated with it. You can optionally create your own `ipyleaflet` maps directly using the tile layers that get generated from an `Image`, but the simplest way to work with the data is to use the built in `wf.map` object, which is a custom instantiation of an ipyleaflet map.

For example, we can visualize our `s2_image` object from above:

In [ ]:
layer = s2_image.pick_bands("red green blue").visualize("rgb layer", scales=[[0,0.4], [0,0.4], [0,0.4]])
wf.map

Note that in the map above we used the `scales=` keyword argument to scale the imagery to be visualized between 0% and 40% reflectance. Oftentimes it may be difficult to know what an appropriate visualization range will be. In those cases, a simple way to scale the imagery is to use the `Autoscale` widget (wand icon) associated with the layer. For example, for the "rgb layer" controls below the map widget, click `Autoscale` button (the wand icon on the right).  After you do so, you should see information that a compute `Job` is running, which calculates the range of values for the current window view, then sets the scale uniformly for the display. For this view, you should see that the rgb ranges of values are now pinned to something like [[0.08-0.3], [0.08, 0.2], [0.1, 0.2]],meaning that there are few pixels outside of the 0-20% reflectance range. If you pan/zoom to different areas and wish to re-scale your images, simply click the wand again!

### Derived Quantities

In the above R/G/B example of Sentinel-2 Imagery, we were performing a simple minimum composite over any of the available data. However, the real power behind the Workflows API is the ability to create derived quantities and visualize the result on-the-fly. For example, if one were interested in the vegetative health of the above area, you might compute something called the Normalized Difference Vegetative Index, or NDVI. NDVI is defined by the ratio of (NIR - Red) / (NIR + Red), where NIR is the Near-infrared wavelength band. When plants are healthy and growing, they absorb much of the red light (for photosynthesis), but reflect the near-infrared (in order to stay cool).  That means NIR is large, Red is small (near zero), and a very healthy plant might have an NDVI value of 1. On the other hand, areas like water have an NDVI of near zero.

Defining a derived quantity is simple. First we're going to pick out the NIR and Red bands using `Image.unpack_bands`, and then define the ndvi using regular python syntax:

In [ ]:
nir_image, red_image = s2_image.unpack_bands("nir red")
ndvi_image = (nir_image - red_image) / (nir_image + red_image)
ndvi_image

This `ndvi_image` variable now references a new `Image` object, just as the `s2_image` object from above, as well as the individual band objects `nir_image` and `red_image`.

That means that we can visualize each of these layers. For this example, it helps to re-center the map. Here we'll target an area similar to above by modifying the `center` attribute of the `wf.map` object. After that, we simply use the `.visualize` method on each variable. Finally we'll display the map again in the next cell. Note that this is the same map object as from above. You may want to use the opacity slider to change the visibility of each individual layer.

Note: In a JupyterLab environment it may be helpful to create a new view from the output cell and split the map off to the right, for example.

In [ ]:
wf.map.center = [36.05, -102.78]
red_image.visualize("Red", scales=[0., 1.], colormap='Reds')
nir_image.visualize("NIR", scales=[0., 1.], colormap='Greens')
ndvi_image.visualize("NDVI", scales=[0., 1.], colormap='viridis')
wf.map

### Parameterized Workflows

The Workflows API also exposes a way to parameterize your models. For this example, we'll create a threshold field based on the value of the NDVI in a given area. To start we create a parameter which is a combination of a name and a data type. Here we create an ndvi_threshold parameter of type wf.Float. Note you need to use the types defined in the workflows package, i.e. `wf.Float`, `wf.Int`, etc.

In [ ]:
ndvi_threshold_param = wf.parameter("ndvi_threshold", wf.Float)
ndvi_threshold_param

This parameter does not yet have a value assigned to it, but we can now use it to define additional derived layers. Here we'll classify everything greater than the threshold as "growing". 

In [ ]:
is_growing = ndvi_image > ndvi_threshold_param

Now we can visualize this layer as usual, but add in a default value for the ndvi_threshold, which is referenced by the name created above, not the name of the variable. That returns a handle that then allows you to edit that parameter. Execute the next two cells, then modify the threshold and watch how the displayed "growing" layer changes.  NDVI is defined such that range is [-1, 1], so avoid going outside of that range with this parameter.

In [ ]:
layer = is_growing.visualize("growing", scales=[0, 1], colormap='magma', ndvi_threshold=0.3)
layer

In [ ]:
wf.map

### Wrap Up

In this guide you've learned:
* Important components of the Workflows API, including `ImageCollection`'s, `Image`'s, and `GeoContext`s.
* How to execute compute `Job`'s over a `GeoContext`
* How to visualize results in a map-based interface
* How to parameterize your Workflow models

Next you might be interested in some of the example workflows in the `/example_notebooks/examples` section of the included notebooks. Otherwise check out additional documentation on [Docs](https://docs.descarteslabs.com).